<a href="https://colab.research.google.com/github/shunya-yan/colab/blob/main/05_RAG%E3%81%AE%E5%BF%9C%E7%94%A8_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.環境構築

In [ ]:
!pip install fastapi langchain langchain_community langchain_openai langchain-qdrant qdrant_client langserve langchain_experimental gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 588.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 k

In [ ]:
import os

#APIキーの登録
os.environ["OPENAI_API_KEY"] = "SyhAAQtvnKDo5v1go8iXK5thk1rS1rDutRW5ojMDGtiA7meDblkKWH3puqV6lNrGtm420w9Sn61rGxGhRoP_jPQ"

# 1.ライブラリのインポートと基本設定

In [ ]:
import json
from fastapi import FastAPI, APIRouter, Body, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse, StreamingResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant
from qdrant_client import QdrantClient
from langserve import add_routes
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_experimental.tools import PythonREPLTool
import gradio as gr
import os
from typing import AsyncIterable, Dict
from langchain.load import dumps, loads

# FastAPI setup
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

# 2.langchainの基本設定

In [ ]:
# modelの指定。今回はGPT4 miniを採用
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    streaming=True,
    openai_api_base="https://api.openai.iniad.org/api/v1/")

# Embedding modelはOpenAIのものを利用
embedding_function = OpenAIEmbeddings(
    openai_api_base="https://api.openai.iniad.org/api/v1/"
)

# ベクトルDBの設定。今回はEmbeddingに時間がかかるのであらかじめ用意したcollectionを利用

qdrant_client = QdrantClient(host="3.145.124.44", port=6333)
qdrant = Qdrant(
    client=qdrant_client,
    embeddings=embedding_function,
    collection_name="aws_documents"
)
# retrieverの定義
retriever = qdrant.as_retriever(search_type="mmr", k=10, verbose=True) # ベクトル検索機能を設定

<ipython-input-4-e5b815363bca>:16: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use QdrantVectorStore instead.
  qdrant = Qdrant(


# 3.Chainとpromptの設定

## 3.1.AWSのサービス設計を返すChainの作成

In [ ]:
# ユーザからの入力に応じて、サービス設計を返すプロンプト
description_prompt = ChatPromptTemplate.from_template("""
マークダウン形式で返答してください。
{input}を構築する際に使用すべきAWSサービスについて詳しく教えてください。
リンクする必要のあるサービス間の関係が理解しやすいように説明してください。
""")
# チェーンの定義
description_chain = description_prompt | model | StrOutputParser()

# チェーンの動作イメージ
print(description_chain.invoke({"input": "ECサイト"}))

# ECサイト構築におけるAWSサービス

ECサイトを構築する際には、さまざまなAWSサービスを組み合わせて使用することが重要です。以下に、主要なサービスとその役割、相互関係について詳しく説明します。

## 1. Amazon EC2 (Elastic Compute Cloud)
- **役割**: サーバーを提供し、アプリケーションをホストします。
- **使用例**: ウェブサーバーやアプリケーションサーバーとして使用。
- **関連サービス**: 
  - **Amazon RDS**: データベースをEC2インスタンスからアクセス。
  - **Amazon S3**: 静的ファイル（画像、CSS、JavaScriptなど）をEC2から配信。

## 2. Amazon S3 (Simple Storage Service)
- **役割**: オブジェクトストレージサービスで、静的ファイルを保存します。
- **使用例**: 商品画像やバックアップデータの保存。
- **関連サービス**:
  - **Amazon CloudFront**: S3に保存されたコンテンツをCDN経由で配信。
  - **Amazon EC2**: EC2からS3にアクセスしてファイルを取得。

## 3. Amazon RDS (Relational Database Service)
- **役割**: リレーショナルデータベースを管理します。
- **使用例**: 商品情報、ユーザー情報、注文履歴などのデータを保存。
- **関連サービス**:
  - **Amazon EC2**: アプリケーションからRDSに接続してデータを操作。
  - **AWS Lambda**: サーバーレスアーキテクチャでRDSにアクセス。

## 4. Amazon CloudFront
- **役割**: コンテンツ配信ネットワーク (CDN) で、静的および動的コンテンツを高速に配信します。
- **使用例**: S3に保存された画像や、EC2でホストされているウェブページを配信。
- **関連サービス**:
  - **Amazon S3**: S3のコンテンツをキャッシュして配信。
  - **Amazon EC2**: EC2のアプリケーションからのレスポンスをキャッシュ

## 3.2.サービス設計をJSON形式で返すChainの作成

In [ ]:
# 必ずJson形式で返すように設定
json_output_parser = JsonOutputParser()
json_template = """
AWSサービスの依存関係を表すJSON構造を作成してください。サービスは配列としてオブジェクトで表現し、各オブジェクトには名前、説明、依存関係を含めてください。
依存関係には依存先の名前、説明、オプションでタイプを含めます。
各サービス間の依存関係が理解できるようにしてください。
依存関係とサービス名のみを知る必要があります。
{format_instructions}
{query}
"""
json_prompt = PromptTemplate(
    template=json_template,
    input_variables=["query"],
    partial_variables={"format_instructions": json_output_parser.get_format_instructions()}
)
# Chainの定義
json_chain = json_prompt | model | json_output_parser
# Chainの動作イメージ
#print(json_structure_chain.invoke({"query": "ECサイト"}))

## 3.3 AWSのドキュメントをもとに回答を生成する RAG Chainの作成

In [ ]:
aws_rag_template = """
{context}の情報に基づいて、{aws_service}を使用する際に念頭に置くべき主要なセキュリティ考慮事項の詳細な分析を提供してください。
各セキュリティ対策について、ドキュメント名、ページ番号、引用を必ず明記し、以下のフォーマットに従って回答してください：
「ドキュメント名: [ドキュメント名] ページ番号: [ページ番号] 引用: 『[引用文]』」。
"""
aws_rag_prompt = ChatPromptTemplate.from_template(aws_rag_template)
# Chainの定義
aws_rag_chain = (
    {"context": retriever, "aws_service": RunnablePassthrough()}
    | aws_rag_prompt
    | model
    | StrOutputParser()
)

#Chainの動作イメージ
print(aws_rag_chain.invoke("EC2"))

Amazon EC2を使用する際に考慮すべき主要なセキュリティ対策について、以下に詳細な分析を提供します。

1. **アクセス管理**
   - ドキュメント名: ec2-ug (1).pdf
   - ページ番号: 2674
   - 引用: 『Connect to the EC2 Serial Console』

   EC2インスタンスへのアクセスを管理することは、セキュリティの基本です。適切なIAMポリシーを設定し、必要な権限のみを付与することで、リスクを軽減できます。

2. **ネットワークセキュリティ**
   - ドキュメント名: ec2-types.pdf
   - ページ番号: 0
   - 引用: 『Instance Types』

   EC2インスタンスを配置するVPC（Virtual Private Cloud）内でのネットワークセキュリティグループの設定が重要です。不要なポートを閉じ、特定のIPアドレスからのアクセスのみを許可することで、外部からの攻撃を防ぎます。

3. **データ暗号化**
   - ドキュメント名: ec2-wg.pdf
   - ページ番号: 1622
   - 引用: 『aws ec2 create-fleet \\ Work with EC2 Fleets』

   EC2インスタンス上のデータは、保存時および転送時に暗号化することが推奨されます。AWSのKMS（Key Management Service）を使用して、データの暗号化キーを管理することができます。

4. **監視とログ管理**
   - ドキュメント名: lambda-dg.pdf
   - ページ番号: 22
   - 引用: 『EC2............................................................................................................................................................1150』

   EC2インスタンスの監視とログ管理は、セキュリティインシデントの早期発見に役立ちます。CloudTrailやCloudWatchを使用して、APIコールやシステムのメトリクスを監

# 4.Fusion RAGの設定

In [ ]:
generate_queries_template = """あなたは1つの入力クエリに基づいて複数の検索クエリを生成する有能なアシスタントです。\n 以下に関連する複数の検索クエリを生成してください: {question} \n 出力 (4つのクエリ):"""
generate_queries_prompt = ChatPromptTemplate.from_template(generate_queries_template)

def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return [x[0] for x in reranked_results[:8]]

generate_queries_chain = (
    {"question": RunnablePassthrough()}
    | generate_queries_prompt
    | model
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

print(generate_queries_chain.invoke({"question": "EC2のセキュリティについて詳しく教えてください"}))


fusion_rag_template = """以下のコンテキストに基づいて質問に答えてください:

{context}

質問: {question}
"""
fusion_rag_prompt = ChatPromptTemplate.from_template(fusion_rag_template)

retrieve_chain = (
    {"original_query": RunnablePassthrough()}
    | generate_queries_chain
    | retriever.map()
    | reciprocal_rank_fusion
)

# Fusion RAGの定義
fusion_rag_chain = (
    {
        "context": retrieve_chain,
        "question": RunnablePassthrough(),
    }
    | fusion_rag_prompt
    | model
    | StrOutputParser()
)
# 動作イメージ
print(fusion_rag_chain.invoke({"question": "EC2のセキュリティについて詳しく教えてください"}))

['1. EC2のセキュリティベストプラクティス', '2. AWS EC2インスタンスのセキュリティグループ設定方法', '3. EC2のセキュリティリスクとその対策', '4. EC2でのデータ保護と暗号化の方法']


<ipython-input-8-8f2e55fb92d3>:14: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


Amazon EC2のセキュリティについては、以下のポイントが重要です。

1. **セキュリティグループ**:
   - EC2インスタンスにはセキュリティグループを設定できます。これにより、インバウンドおよびアウトバウンドのトラフィックを制御します。デフォルトでは、新しいセキュリティグループはすべてのアウトバウンドトラフィックを許可しますが、インバウンドトラフィックを許可するためにはルールを追加する必要があります。

2. **最小特権アクセスの実装**:
   - アクセス許可を付与する際には、必要な最小限の権限のみを付与することが推奨されます。これにより、セキュリティリスクを低減し、エラーや悪意のある行動による影響を最小限に抑えることができます。

3. **データ保護**:
   - EC2インスタンスで扱うデータは、暗号化やバックアップを通じて保護する必要があります。特に機密データを扱う場合は、Amazon Macieなどのサービスを利用してデータの検出と保護を行うことが推奨されます。

4. **インフラストラクチャセキュリティ**:
   - EC2インスタンスのセキュリティは、インフラストラクチャ全体のセキュリティと連携しています。IAM（Identity and Access Management）を使用して、リソースへのアクセスを制御し、ユーザーやグループに適切な権限を付与することが重要です。

5. **セキュリティパッチの適用**:
   - インスタンスを新たに立ち上げた際には、最新のソフトウェアパッチを適用し、ファイアウォールの設定を見直すことが重要です。これにより、将来的なセキュリティ侵害を防ぐことができます。

6. **インスタンスの監視とログ管理**:
   - Amazon CloudWatch Logsを使用して、インスタンスのログを監視し、異常なアクティビティを検出することができます。これにより、セキュリティインシデントの早期発見が可能になります。

これらのポイントを考慮することで、EC2インスタンスのセキュリティを強化し、リスクを低減することができます。


# 5.ストリーミング処理とFastAPIルート設定

In [ ]:
# Streaming処理
async def stream_generator(chain, input_data) -> AsyncIterable[str]:
    async for chunk in chain.astream(input_data):
        yield chunk

# FastAPIのルーティング設定

# description_chainを/description/streamにマッピング
@app.post("/description/stream")
async def stream_description(input: dict):
    return StreamingResponse(stream_generator(description_chain, input), media_type="text/event-stream")

# json_cahinを/json/streamにマッピング
@app.post("/json/stream")
async def stream_json(query: dict):
    return StreamingResponse(stream_generator(json_chain, query), media_type="text/event-stream")

# aws_rag_chainを/rag/streamにマッピング
@app.post("/rag/stream")
async def stream_rag(aws_services: Dict[str, list]):
    async def generate():
        for service in aws_services.get("services", []):
            service_name = service.get("name", "")
            yield f"Service: {service_name}\n"
            async for chunk in aws_rag_chain.astream(service_name):
                yield chunk
            yield "\n\n"

    return StreamingResponse(generate(), media_type="text/event-stream")

# fusion_rag_chainを/fusion-rag/streamにマッピング
@app.post("/fusion-rag/stream")
async def stream_fusion_rag_response(question: str):
    return StreamingResponse(stream_generator(fusion_rag_chain, question), media_type="text/event-stream")

# 6.Gradioから呼ぶ関数の定義

In [ ]:
async def stream_generator(chain, input_data) -> AsyncIterable[str]:
    async for chunk in chain.astream(input_data):
        yield chunk

# Gradio用の非同期関数
async def get_description(input_text: str) -> str:
    result = ""
    async for chunk in stream_generator(description_chain, {"input": input_text}):
        result += chunk
        yield result

async def get_json_structure(input_text: str) -> str:
    result = ""
    async for chunk in stream_generator(json_chain, {"query": input_text}):
        result += chunk
    return json.loads(result)

async def get_rag_response(aws_services: str) -> str:
    result = ""
    services = json.loads(aws_services).get("services", [])
    for service in services:
        service_name = service.get("name", "")
        result += f"Service: {service_name}\n"
        async for chunk in stream_generator(aws_rag_chain, service_name):
            result += chunk
            yield result
        result += "\n\n"
        yield result

async def get_fusion_rag_response(question: str) -> str:
    result = ""
    async for chunk in stream_generator(fusion_rag_chain, question):
        result += chunk
        yield result

# 7.Gradioの画面設定

In [ ]:
demo = gr.Blocks()

with demo:
    gr.Markdown("# CloudSecNavigator")

    with gr.Tab("説明"):
        input_text = gr.Textbox(label="作成したいサービスを入力してください")
        output_text = gr.Markdown(label="説明")
        description_button = gr.Button("説明を取得")
        gr.Examples(
            examples=["ECサイト", "サーバーレスアーキテクチャ", "コンテナ化されたマイクロサービス"],
            inputs=input_text
        )

    with gr.Tab("JSON構造"):
        json_input = gr.Textbox(label="作成したいサービスを入力してください")
        json_output = gr.JSON(label="JSON出力")
        json_button = gr.Button("JSONを生成")
        gr.Examples(
            examples=["ECサイト", "サーバーレスアーキテクチャ", "コンテナ化されたマイクロサービス"],
            inputs=json_input
        )

    with gr.Tab("RAG応答"):
        rag_input = gr.Textbox(label="先ほど作成したJSONを入力してください（'services'配列を含む）")
        rag_output = gr.Markdown(label="セキュリティの考慮事項")
        rag_button = gr.Button("セキュリティ分析を取得")
        gr.Examples(
            examples=[
                '{"services": [{"name": "EC2"}, {"name": "S3"}, {"name": "RDS"}]}',
                '{"services": [{"name": "Lambda"}, {"name": "API Gateway"}, {"name": "DynamoDB"}]}'
            ],
            inputs=rag_input
        )

    with gr.Tab("Fusion RAG"):
        fusion_rag_input = gr.Textbox(label="質問を入力してください")
        fusion_rag_output = gr.Markdown(label="Fusion RAG応答")
        fusion_rag_button = gr.Button("Fusion RAG応答を取得")
        gr.Examples(
            examples=[
                "AWSでのマルチAZ配置の利点は何ですか？",
                "ECSとEKSの違いを説明してください",
                "AWSでのデータ暗号化のベストプラクティスは何ですか？"
            ],
            inputs=fusion_rag_input
        )

    description_button.click(get_description, inputs=input_text, outputs=output_text)
    json_button.click(get_json_structure, inputs=json_input, outputs=json_output)
    rag_button.click(get_rag_response, inputs=rag_input, outputs=rag_output)
    fusion_rag_button.click(get_fusion_rag_response, inputs=fusion_rag_input, outputs=fusion_rag_output)


# 8.アプリケーションの起動

In [ ]:
if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://59e6bf355af7d3a7dc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
